In [35]:
raw_data = [4 , 9 , 6 , 7 , 10 , 
     6 , 8 , 3 , 8 , 7 , 
     1 , 3 , 10 , 10 , 6 , 
     8 , 8 , 9 , 6 , 8 , 
     7 , 7 , 8 , 7 , 10 , 
     7 , 5 , 9 , 8 , 4 , 
     8 , 11 , 4 , 10 , 9 , 
     12 , 12 , 4 , 4 , 5 , 
     6 , 7 , 5 , 4 , 7 , 
     6 , 6 , 4 , 8 , 5]

print(sorted(raw_data))


[1, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 12, 12]


In [36]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [37]:
nums = sorted(raw_data)

x1 = min(nums)
x50 = max(nums)

w = x50 - x1 # Размах выборки

intervals = [x1 + i*w/7 for i in range(8)]  # Интервалы чисел после разбиения

cnts = [0] * 7
nums_by_intervals = {}

for i in range(7):  # В каждый интервал пихаем числа из этого интервала
    nums_by_intervals[i] = []
    for j in nums:
        if (intervals[i] <= j < intervals[i + 1] and i != 6) or \
           (intervals[i] <= j <= intervals[i + 1] and i == 6):
            cnts[i] += 1
            nums_by_intervals[i].append(j)

# print(nums_by_intervals[0])

p = .95  # Коэффицент доверительной вероятности - дан в лабе
alpha = 1 - p

Ms = []
sigmas = []
Ds = []
for i in range(8):
    if i != 7: 
        current_nums = nums_by_intervals[i]
    else:
        current_nums = nums
    if len(current_nums) in [0, 1]:
        Ms.append([None, None])
        Ds.append([None, None])
        sigmas.append([None, None])
        continue

    
    M = np.mean(current_nums)
    sigma = np.std(current_nums)
    f = len(current_nums) - 1 # Количество степеней свободы
    
    # Procentile points function - PPF
    # Страница 17 -> формула
    t_critical = sp.stats.t.ppf(1 - alpha/2, df=f) # Критическое значение распределения Стьюдента при заданном доверии
    # t_{1 - alpha/2}

    M_margin = t_critical * sigma / np.sqrt(len(current_nums)) # Погрешность среднего (Мат ожидания)

    trusted_interval = [M - M_margin, M + M_margin] # Доверительный интервал для среднего
    # print(trusted_interval)

    #Доверительеый интервал для диспресии
    D_interval = [f * sigma**2 / sp.stats.chi2.ppf(1 - alpha/2, df=f), f * sigma**2 / sp.stats.chi2.ppf(alpha/2, df=f)] 

    sigma_interval = list(map(np.sqrt, D_interval))

    Ms.append(trusted_interval)
    Ds.append(D_interval)
    sigmas.append(sigma_interval)

    print(trusted_interval, D_interval, sigma_interval)



[3.458211947934116, 4.097343607621439] [0.0788566774630732, 0.6343517662103462] [0.28081431135729745, 0.7964620306143577]
[5.0, 5.0] [0.0, 0.0] [0.0, 0.0]
[6.257058560579413, 6.8096081060872535] [0.13340676808408042, 0.6190467222348426] [0.36524891250225566, 0.7867952225546636]
[8.0, 8.0] [0.0, 0.0] [0.0, 0.0]
[9.173601333160724, 9.937509777950387] [0.11265239637581885, 0.9062168088719229] [0.33563729884477805, 0.951954205238846]
[10.495632919036032, 12.8377004142973] [0.06024111792929263, 8.777308934490483] [0.2454406607090452, 2.9626523478954603]
[6.2359410624656375, 7.604058937534362] [4.042675060312857, 8.996582207272462] [2.010640460229739, 2.999430313788347]


In [38]:
intervalvals = ''
Mvals = ''
sigmavals = ''
Dvals = ''
for i in range(8):
    if i != 7:
        l = f"{intervals[i]:2.2f}"
        r = f"{intervals[i + 1]:2.2f}"
    else:
        l = f"{intervals[0]:2.2f}"
        r = f"{intervals[-1]:2.2f}"
    intervalvals += f"┃{'['+l:>7}; {r+')' if i < 6 else r+']':<7}"

    if Ms[i][0] is not None:
        l = f"{Ms[i][0]:2.2f}"
        r = f"{Ms[i][1]:2.2f}"
    else:
        l = f"{'-':^3}"
        r = f"{'-':^3}"
    Mvals += f"┃{'['+l:>7}; {r+']':<7}"

    if sigmas[i][0] is not None:
        l = f"{sigmas[i][0]:2.2f}"
        r = f"{sigmas[i][1]:2.2f}"
    else:
        l = f"{'-':^3}"
        r = f"{'-':^3}"
    sigmavals += f"┃{'['+l:>7}; {r+']':<7}"

    if Ds[i][0] is not None:
        l = f"{Ds[i][0]:2.2f}"
        r = f"{Ds[i][1]:2.2f}"
    else:
        l = f"{'-':^3}"
        r = f"{'-':^3}"
    Dvals += f"┃{'['+l:>7}; {r+']':<7}"

print(f"┏{'━'*23}{('┳'+'━'*16)*8}┓")
print(f"┃{'Интервалы':^23}{intervalvals}┃")
print(f"┣{'━'*23}{('╋'+'━'*16)*8}┫")
print(f"┃{'Мат. ожидание':^23}{Mvals}┃")
print(f"┣{'━'*23}{('╋'+'━'*16)*8}┫")
print(f"┃{'Среднее отклонение':^23}{sigmavals}┃")
print(f"┣{'━'*23}{('╋'+'━'*16)*8}┫")
print(f"┃{'Дисперсное отклонение':^23}{Dvals}┃")
print(f"┗{'━'*23}{('┻'+'━'*16)*8}┛")

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃       Интервалы       ┃  [1.00; 2.57)  ┃  [2.57; 4.14)  ┃  [4.14; 5.71)  ┃  [5.71; 7.29)  ┃  [7.29; 8.86)  ┃  [8.86; 10.43) ┃ [10.43; 12.00] ┃  [1.00; 12.00] ┃
┣━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━┫
┃     Мат. ожидание     ┃   [ - ;  - ]   ┃  [3.46; 4.10]  ┃  [5.00; 5.00]  ┃  [6.26; 6.81]  ┃  [8.00; 8.00]  ┃  [9.17; 9.94]  ┃ [10.50; 12.84] ┃  [6.24; 7.60]  ┃
┣━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━┫
┃  Среднее отклонение   ┃   [ - ;  - ]   ┃  [0.28; 0.80]  ┃  [0.00; 0.00]  ┃  [0.37; 0.79]  ┃  [0.00; 0.00]  ┃  [0.34; 0.95]  ┃  [0.25; 2.96]  ┃  [2.01; 3.00]  ┃
┣━━━━━━━━━━━━━━━━━━━━━━━╋━━━

### Проверить гипотезы 

1. $ H_0^{(1)}: m_X = M_0 $  
2. $ H_0^{(2)}: D_X = A_0 $

где:  
- $ M_0 = \bar{x} + 0.5s $  
- $ A_0 = 2s^2 $

#### Первая гипотеза
$ H_0^{(1)}: m_X = \bar{x} + 0.5s $ 

In [44]:
for i in range(7):
    if Ms[i][0] is not None:
        print(f"{i} | {Ms[i][0] <= np.mean(nums_by_intervals[i]) + 0.5 * np.std(nums_by_intervals[i]) <= Ms[i][1]} " +
              f"| {Ms[i][0]:6.4f} <= {np.mean(nums_by_intervals[i]):6.4f} + 0.5 * {np.std(nums_by_intervals[i]):6.4f} = {np.mean(nums_by_intervals[i]) + 0.5 * np.std(nums_by_intervals[i]):6.4f} <= {Ms[i][1]:6.4f}")

1 | True | 3.4582 <= 3.7778 + 0.5 * 0.4157 = 3.9856 <= 4.0973
2 | True | 5.0000 <= 5.0000 + 0.5 * 0.0000 = 5.0000 <= 5.0000
3 | True | 6.2571 <= 6.5333 + 0.5 * 0.4989 = 6.7828 <= 6.8096
4 | True | 8.0000 <= 8.0000 + 0.5 * 0.0000 = 8.0000 <= 8.0000
5 | True | 9.1736 <= 9.5556 + 0.5 * 0.4969 = 9.8040 <= 9.9375
6 | True | 10.4956 <= 11.6667 + 0.5 * 0.4714 = 11.9024 <= 12.8377


#### Вторая гипотеза
$ H_0^{(2)}: D_X = 2s^2 $

In [40]:
for i in range(7):
    if Ds[i][0] is not None:
        print(f"{i} | {Ds[i][0] <= 2 * np.std(nums_by_intervals[i])**2 <= Ds[i][1]} " +
              f"| {Ds[i][0]:6.4f} <= {2 * np.std(nums_by_intervals[i])**2:6.4f} <= {Ds[i][1]:6.4f}")

1 | True | 0.0789 <= 0.3457 <= 0.6344
2 | True | 0.0000 <= 0.0000 <= 0.0000
3 | True | 0.1334 <= 0.4978 <= 0.6190
4 | True | 0.0000 <= 0.0000 <= 0.0000
5 | True | 0.1127 <= 0.4938 <= 0.9062
6 | True | 0.0602 <= 0.4444 <= 8.7773


In [47]:
observed_freqs, _ = np.histogram(nums, bins=intervals)
# print(observed_freqs)

# Cumulative distribution function - просто функция распределения
cdf = sp.stats.norm(np.mean(nums), np.std(nums, ddof=1)).cdf(intervals)
expected = np.diff(cdf) * len(nums)
# print(expected)

xi2 = np.sum((observed_freqs - expected)**2 / expected )
print(xi2)

xi2_critical = sp.stats.chi2.ppf(1 - .05 , 7 - 1 - 2) # df = n - 1 - число оцененных параметров

if xi2 < xi2_critical:
    print(f"Нулевая гипотеза верна: χ² = {xi2:.3f} < {xi2_critical:.3f}")
else:
    print(f"Нулевая гипотеза неверна: χ² = {xi2:.3f} ≥ {xi2_critical:.3f}")

9.225571763405675
Нулевая гипотеза верна: χ² = 9.226 < 9.488
